In [1]:
# Clone and install repo
!git clone https://github.com/tensorflow/tpu.git
!cd tpu/models; pip install -e .

Cloning into 'tpu'...
remote: Enumerating objects: 10182, done.
remote: Counting objects: 100% (384/384), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 10182 (delta 213), reused 293 (delta 166), pack-reused 9798
Receiving objects: 100% (10182/10182), 24.45 MiB | 28.91 MiB/s, done.
Resolving deltas: 100% (7251/7251), done.
Obtaining file:///content/tpu/models
  Running setup.py develop for cloud-tpu-samples


In [2]:
%cd tpu/models/official/efficientnet/lite

/content/tpu/models/official/efficientnet/lite


In [3]:
# Download and extract original weights
!curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/lite/efficientnet-lite0.tar.gz | tar xz
!curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/lite/efficientnet-lite1.tar.gz | tar xz
!curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/lite/efficientnet-lite2.tar.gz | tar xz
!curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/lite/efficientnet-lite3.tar.gz | tar xz
!curl https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/lite/efficientnet-lite4.tar.gz | tar xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 53.5M  100 53.5M    0     0  46.1M      0  0:00:01  0:00:01 --:--:-- 46.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 62.4M  100 62.4M    0     0  36.2M      0  0:00:01  0:00:01 --:--:-- 36.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70.1M  100 70.1M    0     0  41.6M      0  0:00:01  0:00:01 --:--:-- 41.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 94.4M  100 94.4M    0     0  46.1M      0  0:00:02  0:00:02 --:--:-- 46.1M
  % Total    % Received % Xferd  Average Speed   Tim

In [4]:
# Download labelmap and reference image
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.txt
!wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O panda.jpg

--2021-07-29 09:34:29--  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 64.233.170.128, 108.177.11.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30565 (30K) [text/plain]
Saving to: ‘labels_map.txt’

labels_map.txt      100%[===================>]  29.85K  --.-KB/s    in 0s      

2021-07-29 09:34:29 (119 MB/s) - ‘labels_map.txt’ saved [30565/30565]

--2021-07-29 09:34:29--  https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116068 (113K) [image/jpeg]
Saving to: ‘panda.jpg’

panda.jpg 

In [11]:
import tensorflow as tf
import numpy as np

MODELS = [
          ("efficientnet-lite0", 224),
          ("efficientnet-lite1", 240),
          ("efficientnet-lite2", 260),
          ("efficientnet-lite3", 280),
          ("efficientnet-lite4", 300),
]


In [23]:
!rm -r outputs

rm: cannot remove 'outputs': No such file or directory


In [24]:
for model, shape in MODELS:

    !mkdir outputs

    !python ../export_model.py \
      --model_name=$model \
      --ckpt_dir=/content/tpu/models/official/efficientnet/lite/$model/ \
      --output_saved_model_dir outputs/stuff/ \
      --image_size $shape \
      --output_tflite=outputs/stuff.tflite \
      --quantize=False

    loaded = tf.saved_model.load("outputs/stuff")

    img = tf.image.decode_jpeg(tf.io.read_file("panda.jpg"))
    img = tf.image.resize(img, (shape, shape))
    img = (img - 127.00) / 128.00
    img = tf.expand_dims(img, axis=0)

    out = loaded.signatures['serving_default'](img)

    out_name = f"/content/b{model[-1]}_lite_output_{shape}.npy"
    np.save(out_name, out['output'].numpy())


    !rm -r outputs



2021-07-29 09:46:48.659459: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-29 09:46:50.484260: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-29 09:46:50.496293: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-07-29 09:46:50.496351: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (50523cffd032): /proc/driver/nvidia/version does not exist
I0729 09:46:50.501246 140692196018048 efficientnet_lite_builder.py:100] global_params= GlobalParams(batch_norm_momentum=0.99, batch_norm_epsilon=0.001, dropout_rate=0.2, data_format='channels_last', num_classes=1000, width_coefficient=1.0, depth_coefficient=1.0, depth_divisor=8, min_depth=None, survival_prob=0.8, relu_fn=<function relu6 at 0x7ff5

In [25]:
%cd /content/

/content


In [26]:
!tar czvf original_outs_w_preprocessing.tar.gz *.npy

b0_out_224.npy
b1_out_240.npy
b2_out_260.npy
b3_out_280.npy
b4_out_300.npy
